In [8]:
# ============================================================
# CELLULE 1 — Session Spark avec JARs Iceberg chargés
# ============================================================

from pyspark.sql import SparkSession

# Chemin des JARs dans le conteneur spark-master (monté via volume Docker)
JARS = ",".join([
    "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.5.2.jar",
    "/opt/spark/jars/nessie-spark-extensions-3.5_2.12-0.90.4.jar",
    "/opt/spark/jars/hadoop-aws-3.3.4.jar",
    "/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar",
])

spark = (
    SparkSession.builder
    .appName("Jupyter-Iceberg-Nessie")
    .master("spark://spark-master:7077")

    # ── Charger les JARs depuis le master ───────────────────
    .config("spark.jars", JARS)

    # ── Extensions SQL ──────────────────────────────────────
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,"
        "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
    )

    # ── Catalogue Iceberg via Nessie ────────────────────────
    .config("spark.sql.catalog.iceberg",              "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.iceberg.uri",          "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.iceberg.ref",          "main")
    .config("spark.sql.catalog.iceberg.authentication.type", "NONE")
    .config("spark.sql.catalog.iceberg.warehouse",    "s3a://warehouse/")

    # ── Connexion MinIO ─────────────────────────────────────
    .config("spark.hadoop.fs.s3a.endpoint",           "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key",         "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key",         "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access",  "true")
    .config("spark.hadoop.fs.s3a.impl",               "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
    )

    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print(f"✅ Spark {spark.version} connecté — master : {spark.sparkContext.master}")


# ============================================================
# CELLULE 2 — Vérifier les namespaces
# ============================================================

spark.sql("SHOW NAMESPACES IN iceberg").show()
# Attendu : bronze | silver | gold


# ============================================================
# CELLULE 3 — Lire une table Gold
# ============================================================

df = spark.table("gold.daily_sales")
df.orderBy("order_date", ascending=False).show(10)


# ============================================================
# CELLULE 4 — Voyage dans le temps
# ============================================================

# Lister les snapshots
spark.sql("""
    SELECT snapshot_id, committed_at, operation
    FROM iceberg.bronze.orders.snapshots
""").show(5)

# Requête sur un snapshot précis
spark.sql("""
    SELECT COUNT(*) AS nb_commandes
    FROM iceberg.bronze.orders
    VERSION AS OF <snapshot_id>
""").show()


# ============================================================
# CELLULE 5 — Évolution de schéma sans interruption
# ============================================================

spark.sql("""
    ALTER TABLE iceberg.bronze.orders
    ADD COLUMN remise_percent DOUBLE
""")

spark.sql("SELECT order_id, total_amount FROM iceberg.bronze.orders LIMIT 5").show()


# ============================================================
# CELLULE 6 — Élagage de partitions
# ============================================================

# Scan complet
spark.sql("SELECT COUNT(*) AS total FROM iceberg.bronze.orders").show()

# Scan partitionné uniquement (~100x plus rapide sur grands volumes)
spark.sql("""
    SELECT COUNT(*) AS commandes_du_jour
    FROM iceberg.bronze.orders
    WHERE order_date = '2024-02-15'
""").show()

spark.sql("SELECT * FROM iceberg.bronze.orders.partitions").show()

✅ Spark 3.5.0 connecté — master : spark://spark-master:7077


Py4JJavaError: An error occurred while calling o39.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'iceberg': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30)
	at or